In [43]:
#Author: ML Tlachac, WPI
#For StudentSADD, 2021

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from scipy import stats
import collections
import operator
import argparse
import random
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn import metrics
from statistics import mean 
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import load_digits
from sklearn import svm
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.decomposition import PCA, KernelPCA, NMF

In [44]:
#data = pd.read_csv("unscriptedML.csv")
#unscriptedML.csv
#aPromptTextFeatures.csv
#tPromptTextFeatures.csv
print(data.shape)

#binary labels
d10 = []
s1 = []
for i in range(0, data.shape[0]):
    if int(data.phq9[i]) >= 10:
        d10.append(1)
    else:
        d10.append(0)
    if int(data.q9[i]) >= 1:
        s1.append(1)
    else:
        s1.append(0)
data["phq9"] = d10
data["q9"] = s1
data.head()

data.head()

(110, 2271)


,id,phq9,q9,F0final_sma_f0v_duration,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,...,logHNR_sma_de_risetime,F0final_sma_f0v_nnz,F0final_sma_f0v_meanSegLen,F0final_sma_f0v_maxSegLen,F0final_sma_f0v_minSegLen,F0final_sma_f0v_segLenStddev,F0final_sma_f0p_meanSegLen,F0final_sma_f0p_maxSegLen,F0final_sma_f0p_minSegLen,F0final_sma_f0p_segLenStddev
0,8181,0,0,29.72,0.053588,0.121533,0.279239,0.067945,0.157706,0.225651,...,0.428375,0.399394,0.237500,1.20,0.02,0.192370,0.368723,3.18,0.02,0.611432
1,8170,0,0,19.12,0.014378,0.023862,0.076276,0.009483,0.052414,0.061898,...,0.448418,0.288180,0.167742,0.57,0.03,0.124323,0.428387,4.53,0.02,0.821184
2,4041,1,0,29.60,0.054657,0.080013,0.177709,0.025356,0.097696,0.123052,...,0.444540,0.318919,0.238947,0.79,0.03,0.188216,0.519474,3.48,0.02,0.737228
3,8650,1,0,29.82,0.140065,0.247997,0.375071,0.107933,0.127074,0.235007,...,0.432584,0.525486,0.210704,0.58,0.02,0.149160,0.189014,1.94,0.02,0.278452
4,4782,1,0,29.65,0.041752,0.075815,0.195711,0.034062,0.119896,0.153958,...,0.457752,0.326813,0.210455,0.77,0.03,0.154375,0.453023,1.90,0.02,0.491564


In [51]:
#only use this cell for multimodal models or to load scripted audio

audio = pd.read_csv("ScriptedML.csv")
#audio = audio.drop(columns = ["phq9","q9"])

#data = data.merge(audio, on = "id", how = "inner")
#print(data.shape)
data.head()

,id,phq9,q9,F0final_sma_f0v_duration,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,...,logHNR_sma_de_risetime,F0final_sma_f0v_nnz,F0final_sma_f0v_meanSegLen,F0final_sma_f0v_maxSegLen,F0final_sma_f0v_minSegLen,F0final_sma_f0v_segLenStddev,F0final_sma_f0p_meanSegLen,F0final_sma_f0p_maxSegLen,F0final_sma_f0p_minSegLen,F0final_sma_f0p_segLenStddev
0,8181,0,0,29.72,0.053588,0.121533,0.279239,0.067945,0.157706,0.225651,...,0.428375,0.399394,0.237500,1.20,0.02,0.192370,0.368723,3.18,0.02,0.611432
1,8170,0,0,19.12,0.014378,0.023862,0.076276,0.009483,0.052414,0.061898,...,0.448418,0.288180,0.167742,0.57,0.03,0.124323,0.428387,4.53,0.02,0.821184
2,4041,1,0,29.60,0.054657,0.080013,0.177709,0.025356,0.097696,0.123052,...,0.444540,0.318919,0.238947,0.79,0.03,0.188216,0.519474,3.48,0.02,0.737228
3,8650,1,0,29.82,0.140065,0.247997,0.375071,0.107933,0.127074,0.235007,...,0.432584,0.525486,0.210704,0.58,0.02,0.149160,0.189014,1.94,0.02,0.278452
4,4782,1,0,29.65,0.041752,0.075815,0.195711,0.034062,0.119896,0.153958,...,0.457752,0.326813,0.210455,0.77,0.03,0.154375,0.453023,1.90,0.02,0.491564


In [52]:
testids = [1607712777, 292, 2613, 1610640355, 1607494599, 1607040811, 1608492986, 6390, 396, 1607734901, 1607350992, 1608992344, 1609903202, 74, 7159, 4698, 7547, 4441, 1607097951, 8479, 8170, 4707, 7516, 1609174124, 1608853150, 8516, 1611424664, 2843, 1607040596, 1953, 1607772081, 1608564004, 2627, 1607217921, 1607118643, 1607314413, 1609887404, 1608335387, 4098, 1607046006, 1608242917, 8918, 1607131299, 9754, 1607262842, 1607273026, 2478, 1607536408, 1607291545, 1608707232, 1609941585, 1608200497, 1610630377, 7711, 1607810287, 9934, 1608850448, 4041, 1609166629, 1608168856, 1607572897, 6831, 1608586814, 1608588581, 2837, 8180, 1608631410, 1607051003, 3830, 4879, 1608920128, 1607019351, 8181, 3473, 1608335906, 1607738757, 1608770486, 7564, 1607495239, 1609983150, 1607397061, 1607696074, 103, 2222]
trainids = [4769, 1607928177, 1607269923, 7755, 4598, 1607807806, 1608741452, 3323, 1610110670, 1607133044, 9745, 1607291670, 5245, 4442, 319, 1607133218, 1607010270, 1608587203, 1609256130, 1608582258, 5028, 1609771771, 5229, 3517, 1608595561, 1608048050, 1607410780, 528, 1607134906, 3102, 1607555727, 1609887167, 3985, 7256, 3523, 1607289708, 1609890222, 850, 1608917024, 5047, 1608061691, 4782, 1608062276, 1056, 1611517276, 1607636681, 1607891972, 5571, 1609052616, 1607927243, 2525, 4353, 1610818662, 8640, 1607559849, 6706, 1608624428, 1607968838, 1608672132, 552, 1608537399, 1610381937, 1608607986, 381, 1608589576, 3920, 1608059746, 1609027319, 1607357022, 1607691623, 1609899907, 1608470962, 8791, 1610380419, 3064, 1609473849, 1607712704, 1609887249, 1609888813, 1608588103, 1244, 7279, 1607339125, 1607712682, 8472, 1269, 1607045076, 1607365865, 1846, 191, 1811, 1608702785, 1609049435, 5330, 1607257348, 1609890530, 3278, 1608586899, 1607939718, 2430, 1609893292, 60, 1607270186, 6336, 8650, 1608495626, 1608586953, 2121, 1607295286, 896, 1609889389, 1607560754, 6548, 6580, 1607440988, 1609111416, 1607807159, 8663, 1607129044, 6658, 1607799213, 3933, 1608596696, 1608663032, 1610791060, 1607135820, 1607413039, 1607659758, 1608487726, 4859, 1609142183, 1607276888, 7452, 1607368510, 1607266081, 2623, 1608416516, 2128, 3227, 5881, 6510, 1609166843, 7569, 1607712793, 1608850996, 3273, 1607939838, 9986, 3302, 1607206195, 1609082904, 1607510222, 7612, 1607022963, 1607051040, 1607719324, 1608849324, 1607642639, 1607104225, 705, 1608506424, 1608188073, 8018, 8085, 4755, 1611704179, 1607193886, 7007, 3041, 4001, 1552, 1716, 1608053349, 1608572299, 1608051417, 1607712784, 836, 1607929944, 1607795480, 1608200317, 415, 3662, 1610109929, 2496, 8550, 6868, 1608587385, 1608591490, 7370, 4549, 7505, 1879, 1876, 1608003341]

In [53]:
#train/test split
testdata = data[data['id'].isin(testids)]
print(testdata.shape)
traindata = data[data['id'].isin(trainids)]
print(traindata.shape)

(33, 2271)
(65, 2271)


In [54]:
#limit to features
testContent = testdata[testdata.columns[3:]]
print(testContent.shape)
trainContent = traindata[traindata.columns[3:]]
print(trainContent.shape)

(33, 2268)
(65, 2268)


In [55]:
#NEED TO SCALE BEFORE FEATURE SELECTION/REDUCATION

min_max_scaler = preprocessing.MinMaxScaler()  
np_scaled = min_max_scaler.fit_transform(trainContent)
featureSubset = pd.DataFrame(np_scaled)
np_scaled2 =  min_max_scaler.transform(testContent)
testSubset = pd.DataFrame(np_scaled2)
print(featureSubset.shape)
print(testSubset.shape)

(65, 2268)
(33, 2268)


In [56]:
labels = ["phq9", "q9"]
featureEs = ["PCA", "Chi"]
modelTypelist = ["NB", "XG2", "XG3", "XG4", "LR1", "LR2", "SVC1", "SVC2", "SVC3", "SVC4", "kNN1", "kNN3", "kNN5", "kNN7", "kNN9", "RF2g", "RF2e", "RF3g", "RF3e", "RF4g", "RF4e"]

for label in labels:
    
    target = list(traindata[label])
    
    for featureE in featureEs:
    
        featureDF = []
        testDFs = []

        #chi-squared feature selection
        if featureE == "Chi":
            nFeatureList = list(np.arange(1,8,1)) #iterate through number of features to select

            for numberOfFeatures in nFeatureList:
                chisetup = SelectKBest(chi2, k=numberOfFeatures)
                chisetup = chisetup.fit(featureSubset, target) #fit model with training data
                featureSubset2 = chisetup.transform(featureSubset) #apply to training data
                featureSubset2=pd.DataFrame(featureSubset2).assign(target = target) #add target
                featureDF.append(featureSubset2) #save to list for later use
                testSubset2 = chisetup.transform(testSubset) #apply to testing data
                testDFs.append(pd.DataFrame(testSubset2)) #create dataframe and save to list for later use

        #principal component analysis
        elif featureE == "PCA":
            nFeatureList = list(np.arange(1,8,1)) #iterate through number of principal components to store
            for numberOfFeatures in nFeatureList:
                pca = PCA(n_components=numberOfFeatures)
                pca = pca.fit(featureSubset) #fit model with traning data
                X_pca = pca.transform(featureSubset) #apply to training data
                pcaDF = pd.DataFrame(X_pca) #create dataframe of results
                pcaDF = pcaDF.assign(target = target) #add target
                featureDF.append(pcaDF) #save to list for later use
                testSubset2 = pca.transform(testSubset) #apply to testing data
                testDFs.append(pd.DataFrame(testSubset2)) #create dataframe and save to list for later use
                
                
        #create lists to populate
        flist = [] 
        mlist = []
        llist = []
        featureList = []
        f1List = []
        accuracyList = []
        truePosList = []
        trueNegList = []
        falsePosList = []
        falseNegList = []
        predictions = []
        rseed = []
        
        for f in range(0, len(featureDF)):
            print(featureE)
            print(f)
            
            train_phq9 = featureDF[f] #select correct dataframe for training
            X_test = testDFs[f] #select correct dataframe for testing
            
            # upsampling 
            #Count 1s and 0s
            ones = len(train_phq9.loc[train_phq9['target'] == 1])
            zeros = len(train_phq9.loc[train_phq9['target'] == 0])
            if ones >= zeros:
                majority = 1
                minority = 0
            else:
                majority = 0
                minority = 1

            
            # Upsample TrainingSet 
            train_majority = train_phq9[train_phq9.target==majority]
            train_minority = train_phq9[train_phq9.target==minority]

            # Upsample minority class
            train_minority_upsampled = resample(train_minority, 
                                             replace=True,     # sample with replacement
                                             n_samples=len(train_majority),    # to match majority class
                                             random_state=42) # reproducible results

            # Combine majority class with upsampled minority class
            train_phq9 = pd.concat([train_majority, train_minority_upsampled])
            
            #seperate features and target
            y_train = train_phq9["target"] 
            X_train = train_phq9.drop(columns = "target") #remove target from training data
            
            
            print(X_train.head())
            print(X_test.head())
            
            for r in [1,2,4,8,16,32,64,128,256,512]: #random seeds for reproducibility
                for modelType in modelTypelist:

                    #add data to lists
                    llist.append(label)
                    featureList.append(f +1)
                    flist.append(featureE)
                    mlist.append(modelType)

                    #chose model type
                    #chose model type
                    if modelType == "SVC1":
                        clf = svm.SVC(kernel='rbf', random_state=r)
                    elif modelType == "SVC2":
                        clf = svm.SVC(kernel='linear', random_state=r)
                    elif modelType == "SVC3":
                        clf = svm.SVC(kernel='poly', random_state=r)
                    elif modelType == "SVC4":
                        clf = svm.SVC(kernel='sigmoid', random_state=r)
                    elif modelType == "RF2g":
                        clf = RandomForestClassifier(criterion="gini", max_depth=2, random_state=r)
                    elif modelType == "RF2e":
                        clf = RandomForestClassifier(criterion="entropy", max_depth=2, random_state=r)
                    elif modelType == "RF3g":
                        clf = RandomForestClassifier(criterion="gini", max_depth=3, random_state=r)
                    elif modelType == "RF3e":
                        clf = RandomForestClassifier(criterion="entropy", max_depth=3, random_state=r)
                    elif modelType == "RF4g":
                        clf = RandomForestClassifier(criterion="gini", max_depth=4, random_state=r)
                    elif modelType == "RF4e":
                        clf = RandomForestClassifier(criterion="entropy", max_depth=4, random_state=r)
                    elif modelType == "kNN1":
                        clf = KNeighborsClassifier(n_neighbors=1)
                    elif modelType == "kNN3":
                        clf = KNeighborsClassifier(n_neighbors=3)
                    elif modelType == "kNN5":
                        clf = KNeighborsClassifier(n_neighbors=5)
                    elif modelType == "kNN7":
                        clf = KNeighborsClassifier(n_neighbors=7)
                    elif modelType == "kNN9":
                        clf = KNeighborsClassifier(n_neighbors=9)
                    elif modelType == "XG2":
                        clf = xgb.XGBClassifier(max_depth=2, random_state=r)
                    elif modelType == "XG3":
                        clf = xgb.XGBClassifier(max_depth=3, random_state=r)
                    elif modelType == "XG4":
                        clf = xgb.XGBClassifier(max_depth=4, random_state=r)
                    elif modelType == "LR1":
                        clf = LogisticRegression(random_state=r)
                    elif modelType == "LR2":
                        clf = LogisticRegression(random_state=r, penalty = "l2")
                    elif modelType == "NB":
                        clf = GaussianNB()


                    #train model and make predictions
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    #evaluate model
                    conf_mat = confusion_matrix(list(testdata[label]), y_pred)
                    TN = conf_mat[0][0]
                    TP = conf_mat[1][1]
                    FP = conf_mat[0][1]
                    FN = conf_mat[1][0]
                    precision = TP/(TP+FP)
                    sensitivity = TP/(TP+FN)
                    f1 = (2*precision*sensitivity)/(precision + sensitivity)
                    accuracy = (TP+TN)/(TN+TP+FP+FN)

                    #populate lists with results
                    f1List.append(f1)
                    accuracyList.append(accuracy)
                    truePosList.append(TP)
                    trueNegList.append(TN)
                    falsePosList.append(FP)
                    falseNegList.append(FN)
                    predictions.append(y_pred)
                    rseed.append(r)

        #create dataframe to output results
        resultsDF = pd.DataFrame()
        resultsDF["label"] = llist
        resultsDF["Engineering"] = flist
        resultsDF["model"] = mlist
        resultsDF["nFeatures"] = featureList
        resultsDF["F1"] = f1List
        resultsDF["Accuracy"] = accuracyList
        resultsDF["truePos"] = truePosList
        resultsDF["trueNeg"] = trueNegList
        resultsDF["falsePos"] = falsePosList
        resultsDF["falseNeg"] = falseNegList
        resultsDF["predictions"] = predictions
        resultsDF["randomSeed"] = rseed

        resultsDF.to_csv("resultsAll85/Scripted" + featureE + label + ".csv")

PCA
0
          0
3 -5.109159
4 -4.892737
5 -4.688972
6 -6.075516
7 -7.351165
          0
0 -2.982864
1 -5.335472
2 -4.950867
3 -7.992718
4 -5.569734
PCA
1
          0         1
3 -5.109159 -3.753238
4 -4.892737 -3.448511
5 -4.688972 -3.375943
6 -6.075516 -2.976051
7 -7.351165 -2.834562
          0         1
0 -2.982864 -4.205019
1 -5.335472 -3.001037
2 -4.950867 -3.807158
3 -7.992718 -3.006346
4 -5.569734 -2.900340
PCA
2
          0         1         2
3 -5.109159 -3.753239 -0.629525
4 -4.892737 -3.448514 -0.795109
5 -4.688972 -3.375953 -1.617050
6 -6.075516 -2.976054 -1.381249
7 -7.351165 -2.834555 -1.086337
          0         1         2
0 -2.982864 -4.205019 -0.336421
1 -5.335472 -3.001038 -0.990356
2 -4.950867 -3.807156  0.827379
3 -7.992718 -3.006343  0.669584
4 -5.569734 -2.900341  0.042430
PCA
3
          0         1         2         3
3 -5.109159 -3.753237 -0.629523 -1.257811
4 -4.892737 -3.448513 -0.795109 -1.908024
5 -4.688972 -3.375947 -1.616985 -1.954438
6 -6.075516 -2.9

C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: inva

PCA
6
          0         1         2         3         4         5         6
3 -5.109160 -3.753418 -0.627476 -1.265819  1.080956 -0.780611 -0.279333
4 -4.892736 -3.448597 -0.794262 -1.910242 -0.047559 -0.115949  0.001008
5 -4.688977 -3.375816 -1.616734 -1.955374  1.122923  1.114845 -0.201538
6 -6.075517 -2.976108 -1.380626 -1.779378 -0.467646 -0.940848 -0.190986
7 -7.351159 -2.834506 -1.087668 -1.543497  1.398095 -1.682913  0.529369
          0         1         2         3         4         5         6
0 -2.982865 -4.205088 -0.335819 -1.593683  0.148445  1.733987  1.650309
1 -5.335474 -3.000910 -0.990976 -0.828780 -0.046733  0.147160  1.466012
2 -4.950870 -3.807077  0.826997 -0.092742 -0.166769  0.055731  1.036304
3 -7.992720 -3.006320  0.669137  0.654085  2.357568 -0.770269  0.843321
4 -5.569738 -2.900285  0.042125 -0.605500 -0.621138 -1.031175 -0.188276


C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: inva

Chi
0
          0
3  0.000000
4  0.166812
5  0.044242
6  0.095108
7  0.135974
          0
0  0.259511
1  0.864327
2  0.959019
3  0.185150
4  0.097188
Chi
1
          0    1
3  0.000000  0.0
4  0.166812  0.0
5  0.044242  0.0
6  0.095108  0.0
7  0.135974  0.0
          0         1
0  0.259511  0.000000
1  0.864327  0.090909
2  0.959019  0.090909
3  0.185150  0.000000
4  0.097188  0.000000
Chi
2
          0         1    2
3  0.000000  0.041218  0.0
4  0.166812  0.798180  0.0
5  0.044242  0.373133  0.0
6  0.095108  0.321030  0.0
7  0.135974  0.018703  0.0
          0         1         2
0  0.259511  0.787507  0.000000
1  0.864327  0.186580  0.090909
2  0.959019  0.018580  0.090909
3  0.185150  0.046298  0.000000
4  0.097188  0.013907  0.000000
Chi
3
          0         1         2    3
3  0.000000  0.003184  0.041218  0.0
4  0.166812  0.003585  0.798180  0.0
5  0.044242  0.044193  0.373133  0.0
6  0.095108  0.005054  0.321030  0.0
7  0.135974  0.007377  0.018703  0.0
          0         1 

...

C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: inva

Chi
5
          0         1         2         3         4         5
0  0.000109  0.013581  0.028778  0.004367  0.004396  0.000313
1  0.045062  0.000636  0.000396  0.000091  0.000094  0.011446
2  0.161728  0.003485  0.004523  0.002252  0.002252  0.003612
3  0.000000  0.067157  0.069242  0.004354  0.004344  0.000471
4  0.000000  0.006639  0.007092  0.003090  0.003088  0.000473
          0         1         2         3         4         5
0  0.000265  0.001485  0.001156  0.000319  0.000323  0.000315
1  0.001138  0.000036  0.000012  0.000002  0.000002  0.002781
2  0.241975  0.001303  0.002766  0.000052  0.000054  0.000318
3  0.206173  0.001200  0.002994  0.000023  0.000023  0.001655
4  0.002432  0.000046  0.000160  0.000011  0.000011  0.004005


C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: inva

Chi
6
          0         1         2         3         4         5         6
0  0.000109  0.013581  0.014760  0.028778  0.004367  0.004396  0.000313
1  0.045062  0.000636  0.000959  0.000396  0.000091  0.000094  0.011446
2  0.161728  0.003485  0.001923  0.004523  0.002252  0.002252  0.003612
3  0.000000  0.067157  0.009005  0.069242  0.004354  0.004344  0.000471
4  0.000000  0.006639  0.002201  0.007092  0.003090  0.003088  0.000473
          0         1         2         3         4         5         6
0  0.000265  0.001485  0.000915  0.001156  0.000319  0.000323  0.000315
1  0.001138  0.000036  0.000022  0.000012  0.000002  0.000002  0.002781
2  0.241975  0.001303  0.000920  0.002766  0.000052  0.000054  0.000318
3  0.206173  0.001200  0.000015  0.002994  0.000023  0.000023  0.001655
4  0.002432  0.000046  0.000941  0.000160  0.000011  0.000011  0.004005


C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:162: RuntimeWarning: inva